In [1]:
!pip install transformers

In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


C:\Users\anjal\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
pred_step = "file path"

In [ ]:


def extract_keywords(captions, num_keywords=5):
    # Tokenize captions into words
    words = ' '.join(captions).split()

    # Filter out stopwords
    stop_words = set(stopwords.words('english'))
    words = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]

    # Calculate word frequencies
    word_freq = Counter(words)

    # Select the most frequent words as keywords
    keywords = word_freq.most_common(num_keywords)

    return [keyword[0] for keyword in keywords]

# Example usage
captions = predict_step([''])
keywords = extract_keywords(captions)
hashtags = ['#' + keyword for keyword in keywords]
print("Keywords/Hashtags:", hashtags)


In [ ]:
import cv2
from PIL import Image
from transformers import ViTFeatureExtractor, AutoTokenizer, VisionEncoderDecoderModel
import torch

# Load the video
video_path = '/content/sample_data/videos/two.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize model and tokenizer
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Parameters
frame_interval = 30  # Extract every 30 frames
captions = []

# Function to generate captions for images
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

# Process video frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process every 'frame_interval' frame
    if frame_count % frame_interval == 0:
        # Save frame as an image
        cv2.imwrite("frame.jpg", frame)

        # Get caption for the frame
        caption = predict_step(["frame.jpg"])

        # Append caption to the list
        captions.append((frame, caption[0]))

    frame_count += 1

# Release video capture
cap.release()
stored_captions = captions
# Save or display the images with captions
for i, (image, caption) in enumerate(captions):
    cv2.imwrite(f"image_{i}.jpg", image)
    print(f"Caption for image_{i}.jpg: {caption}")


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
nltk.download('punkt')
nltk.download('wordnet')
def extract_keywords(caption):
    # Tokenize the caption
    tokens = word_tokenize(caption.lower())

    # Remove stop words and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Count token frequencies
    token_counts = Counter(tokens)

    # Sort tokens by frequency
    sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)

    # Extract top keywords (let's say top 3)
    top_keywords = [token for token, count in sorted_tokens[:3]]

    return top_keywords

# Extract keywords from each caption and print
for i, (_, caption) in enumerate(stored_captions):
    keywords = extract_keywords(caption)
    print(f"Keywords for image_{i}.jpg: {keywords}")